In [1]:
from rdkit import Chem
import numpy as np
import pandas as pd
from IPython.display import display
import tqdm
import torch
import deepchem as dc

2022-10-27 17:16:19.817866: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-27 17:16:19.928999: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-27 17:16:19.929016: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-27 17:16:19.950651: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-27 17:16:20.497934: W tensorflow/stream_executor/platform/de

In [2]:
from mol_embedding.chembl_mpnn import *

In [3]:
mol_to_embedding(Chem.MolFromSmiles("CCCCCC")).shape, atom_to_embedding(Chem.MolFromSmiles("CCCCCC"), 5).shape

((128,), (64,))

In [4]:
mol_to_embedding(Chem.MolFromSmiles("C")).shape, atom_to_embedding(Chem.MolFromSmiles("C"), 0).shape

((128,), (64,))

# Dump action embeddings for "action_dataset-filtered.csv"

In [5]:
import pandas as pd

csv_path = "datasets/my_uspto/action_dataset-filtered.csv"
dataset = pd.read_csv(csv_path, index_col=0)
print(dataset.shape)

(122448, 17)


In [6]:
%%time

import tqdm
from ChemRL import ChemRlEnv

env = ChemRlEnv()

action_embeddings = []
failures = []

for i in tqdm.tqdm(range(dataset.shape[0])):
    row = dataset.iloc[i][["rsub", "rcen", "rsig", "rsig_cs_indices", "psub", "pcen", "psig", "psig_cs_indices"]]
    try:
        action_embeddings.append(env._action_embedding(row))
    except Exception as e:
        for sub in ["rsub", "psub"]:
            if Chem.MolToSmiles(Chem.AddHs(Chem.MolFromSmiles(row[sub]))) == row[sub]:
                row[sub] = row[sub] + "[H]"
        action_embeddings.append(env._action_embedding(row))
        failures.append(i)

100%|██████████████████████████████████████████████████████| 122448/122448 [2:19:42<00:00, 14.61it/s]

CPU times: user 13h 34min 26s, sys: 1min 39s, total: 13h 36min 5s
Wall time: 2h 19min 43s


In [8]:
assert len(action_embeddings) == dataset.shape[0], "There are some missing embeddings..... :')"

# Hash indexes

In [9]:
action_embedding_hash = list(map(lambda x: hash(" ".join(map(str, x))), action_embeddings))

In [42]:
action_embeddings = np.array(action_embeddings)
action_embedding_hash = np.array(action_embedding_hash)

# Remove hash collisions (some actions that weren't detected as different before)
x, y = np.unique(action_embedding_hash, return_counts=True)

collision_hashes = x[y!=1]

idx_arr = np.isin(action_embedding_hash, collision_hashes, invert=True)

for coll in collision_hashes:
    idx_arr[np.where(action_embedding_hash==coll)[0][0]] = True

assert idx_arr.sum() == action_embedding_hash.shape[0] - y[y!=1].sum() + y[y!=1].shape[0]

In [57]:
# Set embedding hash as index
dataset.index = action_embedding_hash
dataset = dataset.loc[idx_arr]

dataset.to_csv(csv_path)

In [58]:
# Dump the hash->embedding dict
import pickle

hash_to_embedding_map = {action_embedding_hash[i]: action_embeddings[i] for i in range(len(action_embeddings))}

pickle.dump(hash_to_embedding_map, open("datasets/my_uspto/action_embeddings.pickle", 'wb'))